In [1]:
import sys
sys.path.append('D:\wpzhang\Documents\Personal\Python\HandyRec')
import os
os.chdir('D:\wpzhang\Documents\Personal\Python\HandyRec')

In [2]:
from handyrec.models.match import YouTubeDNN
from handyrec.features import DenseFeature, SparseFeature, SparseSeqFeature

In [3]:
import pandas as pd

In [8]:
data_path = "D:/wpzhang/Documents/Personal/Python/HandyRec/handyrec/examples/"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames)

D:\coding\Anaconda\envs\tf2\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
D:\coding\Anaconda\envs\tf2\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
D:\coding\Anaconda\envs\tf2\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [14]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
user_features = [
    DenseFeature('d_1'),
    SparseFeature('s_1',1000,128),
    SparseSeqFeature(SparseFeature('item_id',10000,32),'item_seq',5)
]
item_feature = SparseFeature('item_id',10000,32)